In [ ]:
! pip install datasets transformers[sentencepiece]
!pip install ipython-autotime

# https://huggingface.co/course/chapter7/3?fw=tf

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.54 s (started: 2022-05-30 00:21:37 +00:00)


In [ ]:
! nvidia-smi
! pwd && ls -alh

Mon May 30 00:21:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Masked language modeling based training for domain adjustment

Similar to https://github.com/huggingface/transformers/tree/main/examples/tensorflow/language-modeling

In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


time: 1.46 s (started: 2022-05-30 00:37:12 +00:00)


In [ ]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

time: 3.98 s (started: 2022-05-30 00:22:28 +00:00)


Check the current state of the mask model

In [ ]:
import numpy as np
import tensorflow as tf

text = "Aside from the perks of a [MASK] and medical insurance"
inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> Aside from the perks of a medical and medical insurance
>>> Aside from the perks of a dentist and medical insurance
>>> Aside from the perks of a pharmacy and medical insurance
>>> Aside from the perks of a mortgage and medical insurance
>>> Aside from the perks of a dental and medical insurance
time: 148 ms (started: 2022-05-30 00:22:32 +00:00)


In [ ]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

time: 32.1 s (started: 2022-05-30 00:22:33 +00:00)


In [ ]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))
sample = imdb_dataset["unsupervised"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: If you've seen the classic Roger Corman version starring Vincent Price it's hard to put it out of your head, but you probably should do because this one is totally different. Subtlety has been abandoned in favour of gross-out horror - nudity, gore and all-round unpleasantness. OK it's ridiculous, trashy, sensationalised and historically dubious (did any members of the Inquisition really wear horn-rimmed glasses?), but despite all this it is strangely compelling. I literally couldn't tear myself away from the screen until the end of the movie. If there's a bigger compliment you can pay to a film I don't know what it is.'
'>>> Label: -1'

'>>> Review: For me, this was the most moving film of the decade. Samira Makhmalbaf shows pure bravery and vision in the making. She has an intelligence and gift for speaking to the people, regardless of their nationality or beliefs. I am inspired and touched by her humanity and can only hope that she has touched many people the same way. 

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

time: 45 s (started: 2022-05-30 00:23:05 +00:00)


In [ ]:
# get model context size (BigBird and Longformer have longer contexts)
tokenizer.model_max_length

512

time: 3.53 ms (started: 2022-05-30 00:23:50 +00:00)


In [ ]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'
time: 3.27 ms (started: 2022-05-30 00:23:50 +00:00)


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 800'
time: 2.28 ms (started: 2022-05-30 00:23:50 +00:00)


In [ ]:
"""
- Split the concatenated reviews into chunks of the size given by block_size (chunk_size). 
- Iterate over the features in concatenated_examples and use a list comprehension 
  to create slices of each feature. 
    - The result is a dictionary of chunks for each feature.
"""
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 32'
time: 3.19 ms (started: 2022-05-30 00:23:50 +00:00)


In [ ]:
def group_texts(examples):
    """
    Version of the concat-split logic above that can be applied to the tokenized
    dataset
    """
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

time: 4min 46s (started: 2022-05-30 00:23:50 +00:00)


In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"]), tokenizer.decode(lm_datasets["train"][1]["labels"])

("as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,",
 "as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not s

time: 5.19 ms (started: 2022-05-30 00:28:36 +00:00)


In [ ]:
# utility to mask a random fraction of tokens with [MASK]
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] belong rented i am curious - [MASK] from my video store because of all the controversy that surrounded it [MASK] it was first [MASK] in 1967. i also heard [MASK] [MASK] first it was volunteered by u. anymore. customs if [MASK] ever [MASK] to enter this country [MASK] [MASK] being a fan of films considered " controversial " i really had to see [MASK] for myself. < br / > < br / > [MASK] plot [MASK] centered around a young swedish germanic [MASK] named lena [MASK] wants to learn everything she can about [unused427]. in particular she wants to focus her attentions to making [MASK] sort of documentary on what the average swede [MASK] about certain [MASK] issues [MASK]'

'>>> as the [MASK] war [MASK] [MASK] issues in [MASK] united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics [MASK] she has sex with her drama teacher, classmates [MASK] and married [MASK]. < br / > < narrow / > [MASK] kills [MASK] about i am curious - ye

In [ ]:
# custom data collator to mask whole words instead of tokens
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return tf_default_data_collator(features)

samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)} >> {tokenizer.decode(chunk)}'")


'>>> ['[CLS]', '[MASK]', 'rented', 'i', '[MASK]', 'curious', '-', '[MASK]', 'from', '[MASK]', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', '[MASK]', 'it', 'was', 'first', '[MASK]', 'in', '1967', '.', 'i', 'also', 'heard', 'that', 'at', '[MASK]', 'it', '[MASK]', 'seized', '[MASK]', '[MASK]', '.', 's', '.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', '[MASK]', 'country', '[MASK]', '[MASK]', 'being', 'a', '[MASK]', 'of', 'films', 'considered', '"', '[MASK]', '[MASK]', 'i', 'really', 'had', '[MASK]', '[MASK]', 'this', 'for', 'myself', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'plot', 'is', 'centered', 'around', 'a', 'young', 'swedish', 'drama', '[MASK]', 'named', 'lena', 'who', '[MASK]', 'to', '[MASK]', '[MASK]', 'she', 'can', 'about', 'life', '.', 'in', 'particular', '[MASK]', 'wants', 'to', 'focus', '[MASK]', '[MASK]', '[MASK]', 'to', 'making', 'some', '[MASK]', 'of', '[MASK]', '[MASK]', 'what', 'the', 'average', 'sw'

In [ ]:
# down sample for fast training
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

time: 14.5 ms (started: 2022-05-30 00:28:37 +00:00)


In [ ]:
tf_train_dataset = downsampled_dataset["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = downsampled_dataset["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

time: 190 ms (started: 2022-05-30 00:28:37 +00:00)


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf
import math

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

#lower perplexity score means a better language model
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

model.fit(tf_train_dataset, validation_data=tf_eval_dataset)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0
32/32 [==============================] - 7s 69ms/step - loss: 3.1529
Perplexity: 23.40
312/312 [==============================] - 68s 192ms/step - loss: 2.8488 - val_loss: 2.5590


time: 1min 15s (started: 2022-05-30 00:28:37 +00:00)


In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 2s 68ms/step - loss: 2.6115
Perplexity: 13.62
time: 2.22 s (started: 2022-05-30 00:29:52 +00:00)


In [ ]:
# TODO save to gdrive instead
model.save_pretrained("./my-masked-model")

time: 826 ms (started: 2022-05-30 00:29:54 +00:00)


In [ ]:
from transformers import pipeline

# saniyty check to make sure the newly trained model works
mask_filler = pipeline(
    "fill-mask", model="./my-masked-model", tokenizer=tokenizer,
)
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at ./my-masked-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


>>> aside from the perks of a medical and medical insurance
>>> aside from the perks of a hospital and medical insurance
>>> aside from the perks of a dentist and medical insurance
>>> aside from the perks of a doctor and medical insurance
>>> aside from the perks of a pharmacy and medical insurance
time: 8.31 s (started: 2022-05-30 00:29:55 +00:00)


# QA FineTuning & Training on New Masked Model

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

time: 16 s (started: 2022-05-30 00:30:04 +00:00)


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

time: 4.54 ms (started: 2022-05-30 00:30:20 +00:00)


In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}
time: 3.02 ms (started: 2022-05-30 00:30:20 +00:00)


In [ ]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

  0%|          | 0/88 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

time: 1.92 s (started: 2022-05-30 00:30:20 +00:00)


In [ ]:
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [177, 177, 177]}
{'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."], 'answer_start': [403, 355, 355]}
time: 2.1 ms (started: 2022-05-30 00:30:22 +00:00)


In [ ]:
print(raw_datasets["validation"][2]["context"])
print(raw_datasets["validation"][2]["question"])

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Where did Super Bowl 50 take place?
time: 1.52 ms (started: 2022-05-30 00:30:22 +00:00)


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

time: 2.54 s (started: 2022-05-30 00:30:22 +00:00)


In [ ]:
# verify there is a rust based tokenizer in place
tokenizer.is_fast

True

time: 3.53 ms (started: 2022-05-30 00:30:24 +00:00)


In [1]:
!git clone https://github.com/huggingface/transformers.git
!pushd ./transformers; git checkout v4.19-release; popd

# need to modify line 256 in run_qa.py to log evaluation metrics
# modify line 651 to print eval results
# modify line 634 to save model

Cloning into 'transformers'...
remote: Enumerating objects: 96678, done.
remote: Total 96678 (delta 0), reused 0 (delta 0), pack-reused 96678
Receiving objects: 100% (96678/96678), 89.38 MiB | 30.31 MiB/s, done.
Resolving deltas: 100% (71109/71109), done.
/content/transformers /content
Branch 'v4.19-release' set up to track remote branch 'v4.19-release' from 'origin'.
Switched to a new branch 'v4.19-release'
/content


In [ ]:

# --model_name_or_path ./my-masked-model \
# --model_name_or_path /path/to/distilbert-squad2 \
# --dataset_name squad_v2 \
# --version_2_with_negative \
# --do_eval \
# --per_device_train_batch_size 12 \
# --learning_rate 3e-5 \
# --num_train_epochs 2 \
# --max_seq_length 384 \
# --doc_stride 128 \
!./transformers/examples/tensorflow/question-answering/run_qa.py \
  --output_dir ./my-finetuned-model \
  --dataset_name squad \
  --model_name_or_path distilbert-base-uncased \
  --tokenizer_name distilbert-base-uncased \
  --do_train \
  --do_eval \
  --num_train_epochs 1 \
  --max_train_samples 10 \
  --overwrite_output_dir

05/30/2022 00:40:38 - INFO - __main__ - Training/evaluation parameters TFTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gcp_project=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
inclu

In [ ]:
!ls -alh ./my-finetuned-model

total 278M
drwxr-xr-x 2 root root 4.0K May 30 00:43 .
drwxr-xr-x 1 root root 4.0K May 30 00:33 ..
-rw-r--r-- 1 root root  533 May 30 00:41 config.json
-rw-r--r-- 1 root root  24M May 30 00:43 eval_nbest_predictions.json
-rw-r--r-- 1 root root 993K May 30 00:43 eval_predictions.json
-rw-r--r-- 1 root root 254M May 30 00:41 tf_model.h5
time: 193 ms (started: 2022-05-30 00:43:30 +00:00)


In [ ]:
from transformers import pipeline

# saniyty check to make sure the newly trained model works
q_answerer = pipeline(
    "question-answering", model="./my-finetuned-model", tokenizer=tokenizer,
)
answers = q_answerer(context="I went to eat a cake.", question="eat what?", topk=3)

for pred in answers:
    print(f">>> {pred}")

Some layers from the model checkpoint at ./my-finetuned-model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at ./my-finetuned-model and are newly initialized: ['dropout_76']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:18

>>> {'score': 0.02294921875, 'start': 2, 'end': 9, 'answer': 'went to'}
>>> {'score': 0.021759033203125, 'start': 2, 'end': 15, 'answer': 'went to eat a'}
>>> {'score': 0.0206451416015625, 'start': 2, 'end': 20, 'answer': 'went to eat a cake'}
time: 5.17 s (started: 2022-05-30 00:43:34 +00:00)


In [21]:
from datasets import load_dataset

corpus = load_dataset(
    'csv', 
    data_files=[
      "/content/datasets/undp-jobs-load-ready.csv",
      "/content/datasets/monster-jobs-load-ready.csv",
      "/content/datasets/dice-jobs-load-ready.csv"
    ], 
    column_names=["text"]
)
corpus = corpus["train"].remove_columns('__index_level_0__').train_test_split(test_size=0.1)

Using custom data configuration default-9cf3073f1433db7c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9cf3073f1433db7c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 132796
    })
    test: Dataset({
        features: ['text'],
        num_rows: 14756
    })
})

In [24]:
!head /content/dev-v2.0.jsonl

{"id": "43a12573-cd99-4417-91c7-8d315503cb5b", "title": "2e6fda8d-7b74-455f-a650-b9d5c4522d47", "context": "title middleware developer location vienna va tyson corner terms full time permanent security clearance this position requires a minimum of an active and fully adjudicated secret clearance with the federal government and by virtue us citizenship is required responsibilities designing developing deploying troubleshooting debugging and implementing software code for web and data applications using oracle middleware framework for designing technical solutions based on business processes hands on skills with design architecture development and unit testing of java web based applications and middleware developing solutions in compliance with organizational sdlc processes interface design and architecture participating in architecture reviews application demos and technical reviews prototyping new concepts utilizing big data and open source technologiesworking effectively independently

In [25]:
!du -h /content/dev-v2.0.json

1.1M	/content/dev-v2.0.json


In [27]:
!zip -q -r backup-content.zip /content